# Biopharma

### Install and import packages

In [1]:
!pip install forex_python

from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
from scipy.stats import multivariate_normal

def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")

# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

    print("Mean Vector:\n", mean_vector)
    print("Covariance Matrix:\n", covariance_matrix)
    # The distribution is now defined and can be used for further analysis


Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

In [2]:
samples = mvn_distribution.rvs(size=100, random_state = 42)

print("Sampled draw:", samples)

Sampled draw: [[  5.33903349   0.86624389  73.11981139 111.97694712  20.64765002]
 [  5.61063294   0.92965032  79.34556062 122.17329041  21.55596358]
 [  4.15813507   0.91417896  76.7855405  126.46411362  19.35969562]
 [  4.23125735   0.91322667  76.09887012 128.22232772  18.85391928]
 [  4.22179671   0.86010131  69.36898962  97.33465587  21.08086944]
 [  4.20257082   0.91005302  73.04890421 118.17847894  19.38359263]
 [  5.52254216   0.94084113  81.67286881 127.44813012  20.76991959]
 [  4.54491386   0.92635342  81.90518666 137.20937227  17.19405622]
 [  4.81916288   0.84413823  73.08268079 108.06642289  20.51725975]
 [  4.96882583   0.9037901   77.42010383 130.46304769  19.74663873]
 [  4.8357399    0.88815955  73.90781372 114.47381542  19.15586176]
 [  4.50421436   0.87948083  70.46558563 105.60838233  19.68870145]
 [  4.40187321   0.92911917  77.82142211 126.37336413  18.3082516 ]
 [  4.95103639   0.85179546  69.65778441  99.06923146  21.84976691]
 [  6.09965524   0.80073596  77.01

In [3]:
# Install and import packages
!pip install gurobipy
!pip install tabulate

import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

### Raw Data

In [151]:
selected_yr = 2023
base_yr = 2019

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# Your provided exchange_rate_data
exrate0 = {
    '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
    '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
    '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
    '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
    '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
    '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
}
exrate0 = pd.DataFrame(exrate0 , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

# Minimize cost using Gurobi Binary and Integer optimizer

## Functions to calculate cost, unmet demand, and excess capacity

In [142]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0])
n_lines = range(demand.shape[1]+1)

# Objective function to calculate cost
def calc_total_cost(samples,dec_plant, dec_h, dec_r, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    #reindx = exrate.loc[:, f'{base_yr}'] / exrate.loc[:, f'{selected_yr}']
    reindx = np.append(sample, 1)
    reindex_reshaped = reindx.reshape(-1, 1)  # Reshape for broadcasting
    pcosts_rev = pcosts.values * reindex_reshaped
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost


# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)

    # Handle cases where excess_cap contains None (due to NaN values)
    excess_cap = np.nan_to_num(excess_cap, nan=0.0)

    return excess_cap


# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand


In [170]:

results = []
strategy = []
highcal_flows=[]
relax_flows=[]

for sample in samples:
    # Create a Gurobi model
    model = Model("MinimizeCost")

    # Assign initial value of decision variables
    dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

    # Define decision variables
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

    # Excess Capacity constraints
    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


    # Unmet demand constraints
    unnmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")


    # Update the model
    model.update()

    # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
    model.setObjective(calc_total_cost(sample,dec_plant, dec_h, dec_r, tariff=0), GRB.MINIMIZE)

    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Extract results to print as table
    op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
    op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    highcal_flows.append(op_h)
    relax_flows.append(op_r)



    HighCal_Flow = tabulate(op_h, headers='keys', tablefmt='pretty')
    Relax_Flow = tabulate(op_r, headers='keys', tablefmt='pretty')
    Strategy =tabulate(op_plant, headers='keys', tablefmt='pretty')
    strategy.append(op_plant)
    iteration_result = {
        "HighCal Flow": HighCal_Flow,
        "Relax Flow": Relax_Flow,
        "Strategy": Strategy,
        "Minimum Cost": f"$ {round(model.objVal, 2)} in year {selected_yr}"
    }
    results.append(iteration_result)

# for i, iteration_result in enumerate(results, start=1):
#     # print(f"Iteration {i}")

#     # for key, table_data in iteration_result.items():
#     #     print(f"\n{key}\n")
#     #     print(table_data)

#     print(f"\nMinimum Cost: {iteration_result['Minimum Cost']}\n")

# print(strategy[6])


# sorted_results = sorted(results, key=lambda x: float(x["Minimum Cost"].split()[1]))

# for i, iteration_result in enumerate(sorted_results, start=1):
#     print(f"\nShortlisted Solution {i}:")
#     print(iteration_result["Strategy"])
#     print(f"Minimum Cost: {iteration_result['Minimum Cost']}\n")

# # Print the strategy of the final iteration
# print("\nStrategy of the Final Iteration:")
# print(strategy[-1])

sorted_results = sorted(results, key=lambda x: float(x["Minimum Cost"].split()[1]))

# Extract the first three instances
first_three_results = sorted_results[:3]

for i, iteration_result in enumerate(first_three_results, start=1):
    print(f"\nShortlisted Solution {i}:")
    for key, table_data in iteration_result.items():
         print(f"\n{key}\n")
         print(table_data)

    # print(iteration_result["Strategy"])
    # print(f"Minimum Cost: {iteration_result['Minimum Cost']}\n")

# Print the strategy of the final iteration
print("\nStrategy of the Final Iteration:")





Shortlisted Solution 1:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  0.0   |     0.0     |  7.0  |  0.0   | 4.0  |
| Germany |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  15.0  |     0.0     |  0.0  |  3.0   | 2.0  |
|  U.S.   |     0.0      |  0.0   |     5.0     |  0.0  |  0.0   | 12.0 |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--

In [185]:
import pandas as pd


agg_highcal = pd.concat(highcal_flows).groupby(level=0).sum()
agg_relax = pd.concat(relax_flows).groupby(level=0).sum()

freq_highcal = (agg_highcal > 0).sum(axis=1) / len(highcal_flows)
freq_relax = (agg_relax > 0).sum(axis=1) / len(relax_flows)

freq_highcal_df = pd.DataFrame(freq_highcal, columns=['Frequency'])
freq_relax_df = pd.DataFrame(freq_relax, columns=['Frequency'])

print("Frequency of Use for HighCal Flows:")
print(freq_highcal_df)

print("\nFrequency of Use for Relax Flows:")
print(freq_relax_df)


Frequency of Use for HighCal Flows:
         Frequency
from              
Brazil        0.03
Germany       0.00
India         0.00
Japan         0.00
Mexico        0.03
U.S.          0.03

Frequency of Use for Relax Flows:
         Frequency
from              
Brazil        0.00
Germany       0.06
India         0.00
Japan         0.00
Mexico        0.00
U.S.          0.01


In [190]:
# Strategy 1: High Frequency Use
# Open all plants and supply lines that were used in more than a certain threshold of scenarios, say 50% or your determined significant use threshold.
# Brazil to U.S. and Mexico to Europe are frequently used routes for HighCal and Relax products, ensure dec_h and dec_r for these routes are set to allow flow.


from gurobipy import Model, GRB
import numpy as np


# Mapping of location names to numerical indices
country_indices = {'Brazil': 0, 'Germany': 1, 'India': 2, 'Japan': 3, 'Mexico': 4, 'U.S.': 5}

# Initialize total_costs to store the cost of each simulation
total_costs = []
samples = mvn_distribution.rvs(size=100, random_state = 42)

# Run your simulations
for sample in samples:  # Replace 'samples' with the actual list of samples you're iterating over
    model = Model("MinimizeCost")
    
    # Define decision variables - Ensure this matches your model's requirements
    dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_h_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_r_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}

    # Example to set the lower bound directly for a specific route
    # Convert named routes to indices
    origin_index = country_indices['Brazil']
    destination_index = country_indices['U.S.']
    
    # Set the lower bound directly for the specified routes
    if (origin_index, destination_index) in dec_h:
        dec_h[(origin_index, destination_index)].LB = 1  # Open the route
    if (origin_index, destination_index) in dec_r:
        dec_r[(origin_index, destination_index)].LB = 1  # Open the route
    
    model.update()
    model.setObjective(calc_total_cost(sample,dec_plant, dec_h, dec_r, tariff=0), GRB.MINIMIZE)
    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Record the total cost for this simulation
    total_costs.append(model.objVal)

# Calculate average cost and standard deviation
average_cost = np.mean(total_costs)
cost_std_dev = np.std(total_costs)

print(f"Average Cost: {average_cost}")
print(f"Cost Standard Deviation: {cost_std_dev}")

Average Cost: 1127.145867965407
Cost Standard Deviation: 76.80187159821341


In [191]:
# Strategy 2: Dynamic Adaptation
# For evaluation, choose a configuration that has performed well under a variety of conditions. This could involve a balanced approach, keeping open routes that offer strategic advantages like geographic coverage or potential for cost savings.
# Keep a mix of highly used routes and some that may be strategically important for diversification, even if less frequently used.
highly_used_routes = [('Brazil', 'U.S.'), ('Mexico', 'Germany')]
strategic_routes = [('India', 'Japan')]

# Mapping of location names to numerical indices
country_indices = {'Brazil': 0, 'Germany': 1, 'India': 2, 'Japan': 3, 'Mexico': 4, 'U.S.': 5}
samples = mvn_distribution.rvs(size=100, random_state = 42)

for sample in samples:  # Replace 'samples' with the actual list of samples you're iterating over
    model = Model("MinimizeCost")
    
    # Define decision variables - Ensure this matches your model's requirements
    dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_h_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_r_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}

    # Example to set the lower bound directly for a specific route
    # Convert named routes to indices
    origin_index = country_indices['Brazil']
    destination_index = country_indices['U.S.']
    
    for route in highly_used_routes + strategic_routes:
        origin_name, destination_name = route
        origin_index = country_indices[origin_name]
        destination_index = country_indices[destination_name]
        
        # Set the lower bounds to keep routes open
        # Here we assume dec_h and dec_r use numerical indices as keys, adjust if using names
        dec_h[(origin_index, destination_index)].LB = 1
        dec_r[(origin_index, destination_index)].LB = 1

    model.update()
    model.setObjective(calc_total_cost(sample,dec_plant, dec_h, dec_r, tariff=0), GRB.MINIMIZE)
    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Record the total cost for this simulation
    total_costs.append(model.objVal)

# Calculate average cost and standard deviation
average_cost = np.mean(total_costs)
cost_std_dev = np.std(total_costs)

print(f"Average Cost: {average_cost}")
print(f"Cost Standard Deviation: {cost_std_dev}")
# Keep highly used and strategically important routes open


Average Cost: 2078.0797647168934
Cost Standard Deviation: 958.9384802234727


In [192]:
# Strategy 3: Cost-Saving
# Close plants and supply lines that are rarely used, focusing on maintaining only those with a higher frequency of use.
# India shows very low or no use across scenarios, set dec_plant for India to 0, and similarly, restrict flows from India by setting relevant dec_h and dec_r to 0.

country_indices = {'Brazil': 0, 'Germany': 1, 'India': 2, 'Japan': 3, 'Mexico': 4, 'U.S.': 5}
samples = mvn_distribution.rvs(size=100, random_state = 42)


for sample in samples:  # Replace 'samples' with the actual list of samples you're iterating over
    model = Model("MinimizeCost")
    
    # Define decision variables - Ensure this matches your model's requirements
    dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_h_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"dec_r_{i}_{j}") for i in range(len(locations)) for j in range(len(locations))}
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}

    # Example to set the lower bound directly for a specific route
    # Convert named routes to indices
    india_index = country_indices['India']

    
    for j in range(len(n_lines)):  # or directly use your list of lines if applicable
        dec_plant[(india_index, j)].setAttr("LB", 0)  # Assuming dec_plant uses (country_index, line_index) as keys
    
    # Assuming dec_h and dec_r are dictionaries with keys as tuples indicating routes
    # Restrict flows from India by setting relevant dec_h and dec_r to 0
    # This involves iterating over all possible destination indices
    for dest_index in range(len(country_indices)):  # Assuming all countries are possible destinations
        dec_h[(india_index, dest_index)].setAttr("UB", 0)  # Set upper bound to 0 to restrict flow
        dec_r[(india_index, dest_index)].setAttr("UB", 0)

    model.update()
    model.setObjective(calc_total_cost(sample,dec_plant, dec_h, dec_r, tariff=0), GRB.MINIMIZE)
    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Record the total cost for this simulation
    total_costs.append(model.objVal)

# Calculate average cost and standard deviation
average_cost = np.mean(total_costs)
cost_std_dev = np.std(total_costs)

print(f"Average Cost: {average_cost}")
print(f"Cost Standard Deviation: {cost_std_dev}")

Average Cost: 1726.366825888254
Cost Standard Deviation: 928.4763784323044


In [189]:
#The best model is the ine where Brazil and US line is always open with the lowest cost of $1127.145867965407